# Data ingest information handling

* data submission related information management
  * who, when, what, for which project, data characteristics
* data management related information management
  * ingest, quality assurance, publication, archiving

## Background: approaches at other sites

example workflows in other data centers:
* http://eidc.ceh.ac.uk/images/ingestion-workflow/view
* www.mdpi.com/2220-9964/5/3/30/pdf
* https://www.rd-alliance.org/sites/default/files/03%20Nurnberger%20-%20DataPublishingWorkflows-CollabMtg20151208_V03.pdf
* http://ropercenter.cornell.edu/polls/deposit-data/
* https://www.arm.gov/engineering/ingest
* https://eosweb.larc.nasa.gov/GEWEX-RFA/documents/data_ingest.txt
* https://eosweb.larc.nasa.gov/GEWEX-RFA/documents/how_to_participate.html
* http://www.nodc.noaa.gov/submit/ online tool 

Processing
* https://www2.cisl.ucar.edu/resources/cmip-analysis-platform
  * https://xras-submit-ncar.xsede.org/
  
* http://cmip5.whoi.edu/ 
* https://pypi.python.org/pypi/cmipdata/0.6

## DKRZ ingest workflow system

Data ingest request via
* jupyter notebook on server (http://data-forms.dkrz.de:8080), or
* jupyter notebook filled at home and sent to DKRZ (email, rt system)
  * download form_template from: http://github... 

Data ingest request workflow:
* ingest request is stored in json format in git repo
* all workflow steps are reflected in json subfields
* workflow json format has W3C prov aligned schema and can be transformed to W3C prov format (and visualized as a prov graph) 
* for search git repo can be indexed into a (in-memory) key-value DB

In [2]:
%load_ext autoreload
%autoreload 2

## demo examples 

* load form - get information

In [3]:
## To do: include different examples how to query for info_files based on different properties

info_file = "/home/stephan/tmp/Repos/form_repo/test/test_testsuite_1234.json"

from dkrz_forms import form_handler

my_form = form_handler.load_workflow_form(info_file)

### interactive "help": use ?form.part and tab completion:

In [4]:
?my_form


In [5]:
?my_form.sub

In [11]:
my_form.sub.activity.



 Status message:
-- your submission form Kindermann_test1 was stored in repository 


### add submission check related information 

### add data ingest step related information

### add data quality assurance step related information

In [13]:
sf.status = "submitted"

sf.sub['ticket_id']=22252
sf.sub['checks_done']="cordex form check v0.1"
sf.sub['ticket_url']='https://dm-rt.dkrz.de/Ticket/Display.html?id=22252'
form_handler.form_save(sf)



 Status message:
-- your submission form Kindermann_test1 was stored in repository 


In [14]:
sf.status = "submission_processing"
sf.sub['responsible_person']= "pl"
form_handler.form_save(sf)



 Status message:
-- your submission form Kindermann_test1 was stored in repository 


In [15]:
from datetime import datetime
sf.status = "ingesting"
#sf.check = {}
sf.ing = {'responsible_person': 'pl',
            'started' : str(datetime.now())}
form_handler.form_save(sf)                            
                            



 Status message:
-- your submission form Kindermann_test1 was stored in repository 


In [16]:
sf.status = "ingested"
sf.ing['finished']= str(datetime.now())
sf.ing['target_path']='/scratch/bb0303/data/cordex/test1'
form_handler.form_save(sf)



 Status message:
-- your submission form Kindermann_test1 was stored in repository 


In [17]:
sf.status = "checking"
sf.che = {'responsible_person':'hdh',
          'started':str(datetime.now()),
          'tool_version':'qa_dkrz_v1.1'
            }
form_handler.form_save(sf)        



 Status message:
-- your submission form Kindermann_test1 was stored in repository 


In [21]:
sf.status = 'checked'

sf.che['results']= '/path/to/results'
sf.che['finished']= str(datetime.now())
form_handler.form_save(sf)




 Status message:
-- your submission form Kindermann_test1 was stored in repository 


In [22]:
sf.status = 'publishing'

sf.pub  = {'responsible_person':'kberger',
            'started':str(datetime.now())        
           }
form_handler.form_save(sf)



 Status message:
-- your submission form Kindermann_test1 was stored in repository 


In [23]:
sf.status = 'published'

sf.pub['finished']  = str(datetime.now()) 
sf.pub['search_string'] = "project=cordex&model=hmoc&institute=MPI-M"
           
form_handler.form_save(sf)



 Status message:
-- your submission form Kindermann_test1 was stored in repository 


In [28]:
import json
form_file = open('/home/stephan/tmp/CORDEX/Kindermann_test1.json',"r")
json_info = form_file.read()
#json_info["__type__"] = "sf",
form_file.close()
sf_dict = json.loads(json_info)

{u'status': u'published', u'sub': {u'status': [u'stored'], u'package_name': u'Kindermann_test1.json', u'timestamp': u'2016-03-28 19:25:28.340678', u'ticket_url': u'https://dm-rt.dkrz.de/Ticket/Display.html?id=22252', u'checks_done': u'cordex form check v0.1', u'repo': u'/home/stephan/tmp/CORDEX', u'ticket_id': 22252, u'package_path': u'/home/stephan/tmp/CORDEX/Kindermann_test1.json', u'form_name': u'Kindermann_test1', u'responsible_person': u'pl'}, u'first_name': u'Stephan', u'last_name': u'Kindermann', u'che': {u'started': u'2016-03-28 19:22:40.212107', u'finished': u'2016-03-28 19:24:13.371716', u'tool_version': u'qa_dkrz_v1.1', u'responsible_person': u'hdh', u'results': u'/path/to/results'}, u'keyword': u'test1', u'data_path': u'/scratch/b20030/data/cordex/', u'pub': {u'started': u'2016-03-28 19:24:52.312040', u'search_string': u'project=cordex&model=hmoc&institute=MPI-M', u'finished': u'2016-03-28 19:25:28.340155', u'responsible_person': u'kberger'}, u'institution': u'DKRZ', u'chec